**Use GPU: Runtime -> Change runtime type -> GPU (Hardware Accelerator)**

Setup

In [1]:
!cat ~/.keras/keras.json

{
    "epsilon": 1e-07, 
    "floatx": "float32", 
    "image_data_format": "channels_last", 
    "backend": "tensorflow"
}

In [10]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

Tue Apr 20 04:31:23 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.67       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   48C    P0    38W / 250W |   4715MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [11]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('To enable a high-RAM runtime, select the Runtime > "Change runtime type"')
  print('menu, and then select High-RAM in the Runtime shape dropdown. Then, ')
  print('re-execute this cell.')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 27.4 gigabytes of available RAM

You are using a high-RAM runtime!


Kaggle Dogs-vs-Cats Config

In [6]:
# define the paths to the images directory
IMAGES_PATH = "drive/MyDrive/pyimagesearch/datasets/kaggle-dogs-vs-cats/train" 

# since we do not have validation data or access to the testing
# labels we need to take a number of images from the training
# data and use them instead
NUM_CLASSES = 2
NUM_VAL_IMAGES = 1250 * NUM_CLASSES
NUM_TEST_IMAGES = 1250 * NUM_CLASSES

# define the path to the output training, validation, and testing
# HDF5 files
TRAIN_HDF5 = "drive/MyDrive/pyimagesearch/output/26-kaggle-dogs-vs-cats/hdf5/train.hdf5"
VAL_HDF5 = "drive/MyDrive/pyimagesearch/output/26-kaggle-dogs-vs-cats/hdf5/val.hdf5"
TEST_HDF5 = "drive/MyDrive/pyimagesearch/output/26-kaggle-dogs-vs-cats/hdf5/test.hdf5"

# path to the output model file
MODEL_PATH = "drive/MyDrive/pyimagesearch/output/26-kaggle-dogs-vs-cats/alexnet_dogs_vs_cats.model"

# define the path to the dataset mean
DATASET_MEAN = "drive/MyDrive/pyimagesearch/output/26-kaggle-dogs-vs-cats/dogs_vs_cats_mean.json"

# define the path to the output directory used for storing plots,
# classification reports, etc.
OUTPUT_PATH = "drive/MyDrive/pyimagesearch/output/26-kaggle-dogs-vs-cats/"

Preprocessing

In [2]:
# import the necessary packages
import numpy as np
import cv2
from sklearn.feature_extraction.image import extract_patches_2d
from keras.preprocessing.image import img_to_array

class SimplePreprocessor:
    def __init__(self, width, height, inter=cv2.INTER_AREA):
        # store the target image width, height, and interpolation
        # method used when resizing
        self.width = width
        self.height = height
        self.inter = inter

    def preprocess(self, image):
        # resize the image to a fixed size, ignoring the aspect
        # ratio
        return cv2.resize(image, (self.width, self.height), interpolation=self.inter)

class PatchPreprocessor:
    def __init__(self, width, height):
        # store the target width and height of the image
        self.width = width
        self.height = height

    def preprocess(self, image):
        # extract a random crop from the image with the target width
        # and height
        return extract_patches_2d(image, (self.height, self.width), max_patches=1)[0]

class MeanPreprocessor:
    def __init__(self, rMean, gMean, bMean):
        # store the Red, Green, and Blue channel averages across a
        # training set
        self.rMean = rMean
        self.gMean = gMean
        self.bMean = bMean

    def preprocess(self, image):
        # split the image into its respective Red, Green, and Blue
        # channels
        (B, G, R) = cv2.split(image.astype("float32"))

        # subtract the means for each channel
        R -= self.rMean
        G -= self.gMean
        B -= self.bMean

        # merge the channels back together and return the image
        return cv2.merge([B, G, R])

class ImageToArrayPreprocessor:
    def __init__(self, dataFormat=None):
        # store the image data format
        self.dataFormat = dataFormat

    def preprocess(self, image):
        # apply the Keras utility function that correctly rearranges
        # the dimensions of the image
        return img_to_array(image, data_format=self.dataFormat)

class CropPreprocessor:
    def __init__(self, width, height, horiz=True, inter=cv2.INTER_AREA):
        # store the target image width, height, whether or not
        # horizontal flips should be included, along with the
        # interpolation method used when resizing
        self.width = width
        self.height = height
        self.horiz = horiz
        self.inter = inter

    def preprocess(self, image):
        # initialize the list of crops
        crops = []

        # grab the width and height of the image then use these
        # dimensions to define the corners of the image based
        (h, w) = image.shape[:2]
        coords = [
            [0, 0, self.width, self.height],
            [w - self.width, 0, w, self.height],
            [w - self.width, h - self.height, w, h],
            [0, h - self.height, self.width, h]]

        # compute the center crop of the image as well
        dW = int(0.5 * (w - self.width))
        dH = int(0.5 * (h - self.height))
        coords.append([dW, dH, w - dW, h - dH])

        # loop over the coordinates, extract each of the crops,
        # and resize each of them to a fixed size
        for (startX, startY, endX, endY) in coords:
            crop = image[startY:endY, startX:endX]
            crop = cv2.resize(crop, (self.width, self.height),
                interpolation=self.inter)
            crops.append(crop)

        # check to see if the horizontal flips should be taken
        if self.horiz:
            # compute the horizontal mirror flips for each crop
            mirrors = [cv2.flip(c, 1) for c in crops]
            crops.extend(mirrors)

        # return the set of crops
        return np.array(crops)

HDF5

In [3]:
# import the necessary packages
from keras.utils import np_utils
import numpy as np
import h5py

class HDF5DatasetGenerator:
    def __init__(self, dbPath, batchSize, preprocessors=None,
        aug=None, binarize=True, classes=2):
        # store the batch size, preprocessors, and data augmentor,
        # whether or not the labels should be binarized, along with
        # the total number of classes
        self.batchSize = batchSize
        self.preprocessors = preprocessors
        self.aug = aug
        self.binarize = binarize
        self.classes = classes

        # open the HDF5 database for reading and determine the total
        # number of entries in the database
        self.db = h5py.File(dbPath)
        self.numImages = self.db["labels"].shape[0]

    def generator(self, passes=np.inf):
        # initialize the epoch count
        epochs = 0

        # keep looping infinitely -- the model will stop once we have
        # reach the desired number of epochs
        while epochs < passes:
            # loop over the HDF5 dataset
            for i in np.arange(0, self.numImages, self.batchSize):
                # extract the images and labels from the HDF dataset
                images = self.db["images"][i: i + self.batchSize]
                labels = self.db["labels"][i: i + self.batchSize]

                # check to see if the labels should be binarized
                if self.binarize:
                    labels = np_utils.to_categorical(labels,
                        self.classes)

                # check to see if our preprocessors are not None
                if self.preprocessors is not None:
                    # initialize the list of processed images
                    procImages = []

                    # loop over the images
                    for image in images:
                        # loop over the preprocessors and apply each
                        # to the image
                        for p in self.preprocessors:
                            image = p.preprocess(image)

                        # update the list of processed images
                        procImages.append(image)

                    # update the images array to be the processed
                    # images
                    images = np.array(procImages)

                # if the data augmenator exists, apply it
                if self.aug is not None:
                    (images, labels) = next(self.aug.flow(images,
                        labels, batch_size=self.batchSize))

                # yield a tuple of images and labels
                yield (images, labels)

            # increment the total number of epochs
            epochs += 1

    def close(self):
        # close the database
        self.db.close()

Training Monitor

In [13]:
# import the necessary packages
from keras.callbacks import BaseLogger
import matplotlib.pyplot as plt
import numpy as np
import json
import os

class TrainingMonitor(BaseLogger):
    def __init__(self, figPath, jsonPath=None, startAt=0):
        # store the output path for the figure, the path to the JSON
        # serialized file, and the starting epoch
        super(TrainingMonitor, self).__init__()
        self.figPath = figPath
        self.jsonPath = jsonPath
        self.startAt = startAt

    def on_train_begin(self, logs={}):
        # initialize the history dictionary
        self.H = {}

        # if the JSON history path exists, load the training history
        if self.jsonPath is not None:
            if os.path.exists(self.jsonPath):
                self.H = json.loads(open(self.jsonPath).read())

                # check to see if a starting epoch was supplied
                if self.startAt > 0:
                    # loop over the entries in the history log and
                    # trim any entries that are past the starting
                    # epoch
                    for k in self.H.keys():
                        self.H[k] = self.H[k][:self.startAt]

    def on_epoch_end(self, epoch, logs={}):
        # loop over the logs and update the loss, accuracy, etc.
        # for the entire training process
        for (k, v) in logs.items():
            l = self.H.get(k, [])
            l.append(float(v))
            self.H[k] = l

        # check to see if the training history should be serialized
        # to file
        if self.jsonPath is not None:
            f = open(self.jsonPath, "w")
            f.write(json.dumps(self.H))
            f.close()

        # ensure at least two epochs have passed before plotting
        # (epoch starts at zero)
        if len(self.H["loss"]) > 1:
            # plot the training loss and accuracy
            N = np.arange(0, len(self.H["loss"]))
            plt.figure()
            plt.plot(N, self.H["loss"], label="train_loss")
            plt.plot(N, self.H["val_loss"], label="val_loss")
            plt.plot(N, self.H["accuracy"], label="train_accuracy")
            plt.plot(N, self.H["val_accuracy"], label="val_accuracy")
            plt.title("Training Loss and Accuracy [Epoch {}]".format(len(self.H["loss"])))
            plt.xlabel("Epoch #")
            plt.ylabel("Loss/Accuracy")
            plt.legend()

            # save the figure
            plt.savefig(self.figPath)
            plt.close()

AlexNet

In [14]:
# import the necessary packages
from keras.models import Sequential
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Activation
from keras.layers.core import Flatten
from keras.layers.core import Dropout
from keras.layers.core import Dense
from keras.regularizers import l2
from keras import backend as K

class AlexNet:
    @staticmethod
    def build(width, height, depth, classes, reg=0.0002):
        # initialize the model along with the input shape to be
        # "channels last" and the channels dimension itself
        model = Sequential()
        inputShape = (height, width, depth)
        chanDim = -1

        # if we are using "channels first", update the input shape
        # and channels dimension
        if K.image_data_format() == "channels_first":
            inputShape = (depth, height, width)
            chanDim = 1

        # Block #1: first CONV => RELU => POOL layer set
        model.add(Conv2D(96, (11, 11), strides=(4, 4), input_shape=inputShape, padding="same", kernel_regularizer=l2(reg)))
        model.add(Activation("relu"))
        model.add(BatchNormalization(axis=chanDim))
        model.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2)))
        model.add(Dropout(0.25))

        # Block #2: second CONV => RELU => POOL layer set
        model.add(Conv2D(256, (5, 5), padding="same", kernel_regularizer=l2(reg)))
        model.add(Activation("relu"))
        model.add(BatchNormalization(axis=chanDim))
        model.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2)))
        model.add(Dropout(0.25))

        # Block #3: CONV => RELU => CONV => RELU => CONV => RELU
        model.add(Conv2D(384, (3, 3), padding="same", kernel_regularizer=l2(reg)))
        model.add(Activation("relu"))
        model.add(BatchNormalization(axis=chanDim))
        model.add(Conv2D(384, (3, 3), padding="same", kernel_regularizer=l2(reg)))
        model.add(Activation("relu"))
        model.add(BatchNormalization(axis=chanDim))
        model.add(Conv2D(256, (3, 3), padding="same", kernel_regularizer=l2(reg)))
        model.add(Activation("relu"))
        model.add(BatchNormalization(axis=chanDim))
        model.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2)))
        model.add(Dropout(0.25))

        # Block #4: first set of FC => RELU layers
        model.add(Flatten())
        model.add(Dense(4096, kernel_regularizer=l2(reg)))
        model.add(Activation("relu"))
        model.add(BatchNormalization())
        model.add(Dropout(0.5))

        # Block #5: second set of FC => RELU layers
        model.add(Dense(4096, kernel_regularizer=l2(reg)))
        model.add(Activation("relu"))
        model.add(BatchNormalization())
        model.add(Dropout(0.5))

        # softmax classifier
        model.add(Dense(classes, kernel_regularizer=l2(reg)))
        model.add(Activation("softmax"))

        # return the constructed network architecture
        return model

Train AlexNet

In [15]:
# import the necessary packages
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
import matplotlib
import json
import os

In [16]:
def train_alexnet_dogs_vs_cats():
    # construct the training image generator for data augmentation
    aug = ImageDataGenerator(rotation_range=20, zoom_range=0.15,
        width_shift_range=0.2, height_shift_range=0.2, shear_range=0.15,
        horizontal_flip=True, fill_mode="nearest")

    # load the RGB means for the training set
    means = json.loads(open(DATASET_MEAN).read())

    # initialize the image preprocessors
    sp = SimplePreprocessor(227, 227)
    pp = PatchPreprocessor(227, 227)
    mp = MeanPreprocessor(means["R"], means["G"], means["B"])
    iap = ImageToArrayPreprocessor()

    # initialize the training and validation dataset generators
    trainGen = HDF5DatasetGenerator(TRAIN_HDF5, 128, aug=aug, preprocessors=[pp, mp, iap], classes=2)
    valGen = HDF5DatasetGenerator(VAL_HDF5, 128, preprocessors=[sp, mp, iap], classes=2)

    # initialize the optimizer
    print("[INFO] compiling model...")
    opt = Adam(lr=1e-3)
    model = AlexNet.build(width=227, height=227, depth=3, classes=2, reg=0.0002)
    model.compile(loss="binary_crossentropy", optimizer=opt, metrics=["accuracy"])

    # construct the set of callbacks
    path = os.path.sep.join([OUTPUT_PATH, "{}.png".format(os.getpid())])
    callbacks = [TrainingMonitor(path)]

    # train the network
    model.fit_generator(
        trainGen.generator(),
        steps_per_epoch=trainGen.numImages // 128,
        validation_data=valGen.generator(),
        validation_steps=valGen.numImages // 128,
        epochs=75,
        max_queue_size=10,
        callbacks=callbacks, verbose=1)

    # save the model to file
    print("[INFO] serializing model...")
    model.save(MODEL_PATH, overwrite=True)

    # close the HDF5 datasets
    trainGen.close()
    valGen.close()
    print("[INFO] done.")

In [ ]:
train_alexnet_dogs_vs_cats()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:20: H5pyDeprecationWarning: The default file mode will change to 'r' (read-only) in h5py 3.0. To suppress this warning, pass the mode you need to h5py.File(), or set the global default h5.get_config().default_file_mode, or set the environment variable H5PY_DEFAULT_READONLY=1. Available modes are: 'r', 'r+', 'w', 'w-'/'x', 'a'. See the docs for details.


[INFO] compiling model...


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/75
156/156 [==============================] - 623s 4s/step - loss: 3.7105 - accuracy: 0.5470 - val_loss: 2.8314 - val_accuracy: 0.5942
Epoch 2/75
156/156 [==============================] - 621s 4s/step - loss: 2.5192 - accuracy: 0.6189 - val_loss: 1.9226 - val_accuracy: 0.6850
Epoch 3/75
156/156 [==============================] - 620s 4s/step - loss: 1.8648 - accuracy: 0.6727 - val_loss: 1.6188 - val_accuracy: 0.6530
Epoch 4/75
156/156 [==============================] - 622s 4s/step - loss: 1.4489 - accuracy: 0.7155 - val_loss: 1.1524 - val_accuracy: 0.7800
Epoch 5/75
156/156 [==============================] - 618s 4s/step - loss: 1.1728 - accuracy: 0.7413 - val_loss: 1.0832 - val_accuracy: 0.7081
Epoch 6/75
156/156 [==============================] - 628s 4s/step - loss: 0.9880 - accuracy: 0.7840 - val_loss: 0.8450 - val_accuracy: 0.8026
Epoch 7/75
156/156 [==============================] - 628s 4s/step - loss: 0.8721 - accuracy: 0.7954 - val_loss: 0.8741 - val_accuracy: 0.7410

Rank-5 Accuracy

In [ ]:
import numpy as np

def rank5_accuracy(preds, labels):
	# initialize the rank-1 and rank-5 accuracies
	rank1 = 0
	rank5 = 0

	# loop over the predictions and ground-truth labels
	for (p, gt) in zip(preds, labels):
		# sort the probabilities by their index in descending
		# order so that the more confident guesses are at the
		# front of the list
		p = np.argsort(p)[::-1]

		# check if the ground-truth label is in the top-5
		# predictions
		if gt in p[:5]:
			rank5 += 1

		# check to see if the ground-truth is the #1 prediction
		if gt == p[0]:
			rank1 += 1

	# compute the final rank-1 and rank-5 accuracies
	rank1 /= float(len(preds))
	rank5 /= float(len(preds))

	# return a tuple of the rank-1 and rank-5 accuracies
	return (rank1, rank5)

Crop Accuracy

In [ ]:
# import the necessary packages
from keras.models import load_model
import numpy as np
import progressbar
import json

In [ ]:
def crop_accuracy():
    # load the RGB means for the training set
    means = json.loads(open(DATASET_MEAN).read())

    # initialize the image preprocessors
    sp = SimplePreprocessor(227, 227)
    mp = MeanPreprocessor(means["R"], means["G"], means["B"])
    cp = CropPreprocessor(227, 227)
    iap = ImageToArrayPreprocessor()

    # load the pretrained network
    print("[INFO] loading model...")
    model = load_model(MODEL_PATH)

    # initialize the testing dataset generator, then make predictions on
    # the testing data
    print("[INFO] predicting on test data (no crops)...")
    testGen = HDF5DatasetGenerator(config.TEST_HDF5, 64, preprocessors=[sp, mp, iap], classes=2)
    predictions = model.predict_generator(testGen.generator(), steps=testGen.numImages // 64, max_queue_size=10)

    # compute the rank-1 and rank-5 accuracies
    (rank1, _) = rank5_accuracy(predictions, testGen.db["labels"])
    print("[INFO] rank-1: {:.2f}%".format(rank1 * 100))
    testGen.close()

    # re-initialize the testing set generator, this time excluding the
    # `SimplePreprocessor`
    testGen = HDF5DatasetGenerator(config.TEST_HDF5, 64, preprocessors=[mp], classes=2)
    predictions = []

    # initialize the progress bar
    widgets = ["Evaluating: ", progressbar.Percentage(), " ", progressbar.Bar(), " ", progressbar.ETA()]
    pbar = progressbar.ProgressBar(maxval=testGen.numImages // 64, widgets=widgets).start()

    # loop over a single pass of the test data
    for (i, (images, labels)) in enumerate(testGen.generator(passes=1)):
        # loop over each of the individual images
        for image in images:
            # apply the crop preprocessor to the image to generate 10
            # separate crops, then convert them from images to arrays
            crops = cp.preprocess(image)
            crops = np.array([iap.preprocess(c) for c in crops], dtype="float32")

            # make predictions on the crops and then average them
            # together to obtain the final prediction
            pred = model.predict(crops)
            predictions.append(pred.mean(axis=0))

        # update the progress bar
        pbar.update(i)

    # compute the rank-1 accuracy
    pbar.finish()
    print("[INFO] predicting on test data (with crops)...")
    (rank1, _) = rank5_accuracy(predictions, testGen.db["labels"])
    print("[INFO] rank-1: {:.2f}%".format(rank1 * 100))
    testGen.close()

In [ ]:
crop_accuracy()